In [2]:
##################################################################
#  Цель - отсканировать существующие препринты на сайте Банка России 
#     и собрать из этих них информацию о ключевых словах
# 1) Нужно научиться сканировать сайты и открывать пдф-файлы на этом сайте
# 2) Нужно научиться извлекать текст из этих pdf-файлов по заданному правилу
# 3) Нужно извлеченный текст соотнести с названием конкретного исследования в виде таблицы
# 4) Нужно экспортировать полученные данные в эксель формат
#
# Сайт 1: https://www.cbr.ru/ec_research/ser/
# Сайт 2: https://www.cbr.ru/dkp/system_p/
#################################################################

In [3]:
# библиотеки
import requests
import lxml
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
# целевые сайты
url1 = 'https://www.cbr.ru/ec_research/ser/'
url2 = 'https://www.cbr.ru/dkp/system_p/research_notes/#a_119347'

r1 = requests.get(url1)
r2 = requests.get(url2)

# чтобы начать парсить, нужно преобразовать информацию с сайта в объект "Soup"
soup1 = BeautifulSoup(r1.text, 'html')
soup2 = BeautifulSoup(r2.text, 'html')

In [ ]:
# находим все ссылки на исследования, которые расположены на сайте номер 1
base1 = soup1.find_all('a', class_ = 'referenceable')
# находим все ссылки на исследования, которые расположены на сайте номер 2
base2 = soup2.find_all('a')


In [7]:
# создаем словарь
research_data = []

In [8]:
# извлекаем ссылки и соответствующие названия к статьям из сайта номер 1
for link in base1:
    # извлекаем название и ссылку
    title_span = link.find('span', class_='document-regular_name_visible')
    if title_span:
        title = title_span.get_text(strip=True)
        href = link.get('href')

        research_data.append({
            'title': title,
            'link': f"https://www.cbr.ru{href}"
        })

In [9]:
# извлекаем ссылки и соответствующие названия к статьям из сайта номер 2
for link in base2:
    # извлекаем значение атрибута href из ссылки
    href = link.get('href', '')
    if href and href.startswith('/StaticHtml') or href.startswith('/Content') :
        title = link.get_text(strip=True)
        href = link.get('href')

        research_data.append({
            'title': title,
            'link': f"https://www.cbr.ru{href}"
        })

In [10]:
# можно посмотреть на отобранные статьи
for i, research in enumerate(research_data, 1):
    print(f"{i}. {research['title']}")
    print(f"Ссылка: {research['link']}")
    print()

1. Оценка разрыва и потенциального выпуска с учетом финансовых условий по данным опросов Банка России на примере ЦФО
Ссылка: https://www.cbr.ru/ec_research/ser/wp_155/

2. Учебная модель малой открытой экономики для анализа денежно-кредитной политики (с примерами из практики Банка России)
Ссылка: https://www.cbr.ru/ec_research/ser/wp_154/

3. Жесткости отечественного рынка труда в период структурной трансформации экономики: неизвестные возможности известного источника данных
Ссылка: https://www.cbr.ru/ec_research/ser/wp_153/

4. Неоднородность инфляционных ожиданий разных типов экономических агентов: выводы для денежно-кредитной политики
Ссылка: https://www.cbr.ru/ec_research/ser/wp_152/

5. О равновесиях в модели рынков депозитов с экзогенными издержками перехода вкладчиков из банка в банк
Ссылка: https://www.cbr.ru/ec_research/ser/wp_151/

6. Синхронизация деловых циклов России и Китая
Ссылка: https://www.cbr.ru/ec_research/ser/wp_150/

7. Декомпозиция индекса потребительских цен на 

### Теперь есть список статей и ссылки на них. Требуется из всех ссылок извлечь те, которые ведут на сайт, и заменить их на ссылки на pdf файлы

In [11]:
def replace_with_pdf(articles_list): # данная функция заменяет ссылки на статьи на прямые ссылки на pdf-файлы
    results = articles_list.copy() # финальный словарь, в который будут сохраняться финальные ссылки на pdf

    for i, article in enumerate(results):
        current_link = article['link'] # получаем текущую ссылку на статью
        print(f"Обрабатываем: {current_link}")

        if current_link.lower().endswith('.pdf'):
            print("Уже PDF, пропускаем")
            continue
        
        else:
            print("Уже исправляю ссылку на PDF")
            urltest = current_link
            rtest = requests.get(urltest)
            souptest = BeautifulSoup(rtest.text, 'html.parser')
            basetest = souptest.find_all('a')

            for i, link in enumerate(basetest, 1):
                href = link.get('href')
                if href and href.endswith('.pdf'):
                    if href.startswith('/'):
                        hrefpdf = f"https://www.cbr.ru{href}"
                        article['link'] = hrefpdf
                    elif href.startswith('http'):
                        hrefpdf = href
                        article['link'] = hrefpdf
                    else:
                        hrefpdf = f"https://www.cbr.ru/{href}"
                        article['link'] = hrefpdf

    seen_links = set()
    unique_research = []
    
    for research in results:
        link = research.get('link', '').strip()
        if link.endswith('.pdf'): # есть ссылки, которые ведут не на pdf, а на .xsls
            if link and link not in seen_links:
                seen_links.add(link)
                unique_research.append(research)
            elif not link:
                # Исследования без ссылки тоже добавляем
                unique_research.append(research)
        
    # Возвращаем список уникальных исследований
    return unique_research

research_data_pdf = replace_with_pdf(research_data)

Обрабатываем: https://www.cbr.ru/ec_research/ser/wp_155/
Уже исправляю ссылку на PDF
Обрабатываем: https://www.cbr.ru/ec_research/ser/wp_154/
Уже исправляю ссылку на PDF
Обрабатываем: https://www.cbr.ru/ec_research/ser/wp_153/
Уже исправляю ссылку на PDF
Обрабатываем: https://www.cbr.ru/ec_research/ser/wp_152/
Уже исправляю ссылку на PDF
Обрабатываем: https://www.cbr.ru/ec_research/ser/wp_151/
Уже исправляю ссылку на PDF
Обрабатываем: https://www.cbr.ru/ec_research/ser/wp_150/
Уже исправляю ссылку на PDF
Обрабатываем: https://www.cbr.ru/ec_research/ser/wp_149/
Уже исправляю ссылку на PDF
Обрабатываем: https://www.cbr.ru/ec_research/ser/wp_148/
Уже исправляю ссылку на PDF
Обрабатываем: https://www.cbr.ru/ec_research/ser/wp_147/
Уже исправляю ссылку на PDF
Обрабатываем: https://www.cbr.ru/ec_research/ser/wp_146/
Уже исправляю ссылку на PDF
Обрабатываем: https://www.cbr.ru/ec_research/ser/wp_145/
Уже исправляю ссылку на PDF
Обрабатываем: https://www.cbr.ru/ec_research/ser/wp_144/
Уже испр

In [12]:
# можно посмотреть на отобранные статьи
for i, research in enumerate(research_data_pdf, 1):
    print(f"{i}. {research['title']}")
    print(f"Ссылка: {research['link']}")
    print()

1. Оценка разрыва и потенциального выпуска с учетом финансовых условий по данным опросов Банка России на примере ЦФО
Ссылка: https://www.cbr.ru/StaticHtml/File/180764/wp_155.pdf

2. Учебная модель малой открытой экономики для анализа денежно-кредитной политики (с примерами из практики Банка России)
Ссылка: https://www.cbr.ru/StaticHtml/File/180505/wp_154.pdf

3. Жесткости отечественного рынка труда в период структурной трансформации экономики: неизвестные возможности известного источника данных
Ссылка: https://www.cbr.ru/StaticHtml/File/178855/wp_153.pdf

4. Неоднородность инфляционных ожиданий разных типов экономических агентов: выводы для денежно-кредитной политики
Ссылка: https://www.cbr.ru/StaticHtml/File/178678/wp_152.pdf

5. О равновесиях в модели рынков депозитов с экзогенными издержками перехода вкладчиков из банка в банк
Ссылка: https://www.cbr.ru/StaticHtml/File/177177/wp_151.pdf

6. Синхронизация деловых циклов России и Китая
Ссылка: https://www.cbr.ru/StaticHtml/File/176714

## Извлекаем ключевые слова из PDF

In [13]:
import PyPDF2
from io import BytesIO
import re
import pdfplumber

### *Напишем функцию, которая извлекает текст из pdf-файла*

In [42]:
def extract_text_from_pdf(pdf_link: str) -> str:
    try:
        response = requests.get(pdf_link)
        response.raise_for_status()
                
        with pdfplumber.open(BytesIO(response.content)) as pdf:
            text = []
            total_pages = len(pdf.pages)
            for i in range(min(10, total_pages)):
                page = pdf.pages[i]  # Получаем объект страницы по индексу
                text.append(page.extract_text())

            # Извлекаем последние 4 страницы (если есть)
            if total_pages > 10:  # Чтобы не дублировать страницы
                start_last_pages = max(10, total_pages - 4)  # Начинаем с max(8, total_pages-4)
                for i in range(start_last_pages, total_pages):
                    page = pdf.pages[i]  # Получаем объект страницы по индексу
                    text.append(page.extract_text())
        return text

    except:
        response = requests.get(pdf_link)
        response.raise_for_status()

        pdf_data = BytesIO(response.content)

    
        reader = PyPDF2.PdfReader(pdf_data, strict=False)
        text = []

        total_pages = len(reader.pages)
        
        # Извлекаем первые 10 страниц
        for page_num in range(min(10, total_pages)):
            content = reader.pages[page_num].extract_text()
            text.append(content)
        
        # Извлекаем последние 4 страницы (если есть)
        if total_pages > 10:  # Чтобы не дублировать страницы
            start_last_pages = max(10, total_pages - 4)
            for page_num in range(start_last_pages, total_pages):
                content = reader.pages[page_num].extract_text()
                text.append(content)
            
        return text


### *Напишем функцию, которая очищает текст*

In [15]:
def clean_keywords_text(text: str) -> str: # данная функция очищает и восстанавливает текст
    combined_text = "".join(text) # объединяем все страницы вместе
    lines = combined_text.split('\n') # разделяем текст на строки

    cleaned_lines = []
    for line in lines:
            # Убираем пробелы в начале и конце строки
            line = line.strip()
            cleaned_lines.append(line)

    full_text = ' '.join(cleaned_lines)

    full_text = re.sub(r'(\w+)-\s+', r'\1', full_text)
    full_text = re.sub(r'\s+', ' ', full_text)
    full_text = re.sub(r'\s+:', ':', full_text) # убираем пробел перед двоеточием
    full_text = re.sub(r'\s+ь', 'ь', full_text) # убираем пробел мягким знаком (в некоторых файлах при извлечении текста есть данная проблема)
    full_text = re.sub(r'\s+ы', 'ы', full_text)
    full_text = re.sub(r'\s+ъ', 'ъ', full_text)
    full_text = re.sub(r'\s+-', '-', full_text)
    full_text = re.sub(r'\s+»', '»', full_text)
    full_text = re.sub(r'\s+;', ';', full_text)
    full_text = re.sub(r':(\w)', r': \1', full_text) # добавляем пробел после двоеточия, если его нет

    return full_text.strip()

### *Напишем функцию, которая ищет в тексте ключевые слова*

In [16]:
def extract_keywords(text: str) -> str: # извлечение ключевых слов с учетом различных форматов:
    # варианты паттернов для поиска начала ключевых слов
    patterns = [
        r'Ключевые слова:*',
        r'Keywords:*',
        r'Key words:*',
        r'KEYWORDS:*',
        r'KEY WORDS:*',
        r'Ключевые слова\s*',  # Без двоеточия
        r'Keywords\s*',        # Без двоеточия
        r'Key words\s*',       # Без двоеточия
    ]
    
    for pattern in patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            start_pos = match.end()
            # Находим позицию начала ключевых слов
            keywords_text = text[start_pos:]
            
            # Разбиваем текст на токены для анализа
            tokens = []
            current_token = ""
            
            # Проходим по каждому символу для анализа структуры
            for char in keywords_text:
                if char in [':', '.']:  # Сначала проверяем точку и двоеточие (если они есть, то значит ключевые слова 100% уже перечислены)
                    if current_token:
                        tokens.append(current_token)  # Добавляем последнее слово
                    break  # Выходим из цикла

                elif char in ['ь', 'ы', 'ъ']:
                    if current_token:
                        tokens.append(current_token)
                        current_token = ""
                    tokens.append(char)

                elif char in [',', ';'] or char.isspace():
                    if current_token:
                        tokens.append(current_token)
                        current_token = ""
                    tokens.append(char)
                else:
                    current_token += char


            # Собираем ключевые слова до условия остановки
            result_tokens = []
            stop = False
            prev_comma = False  # Флаг, что предыдущий токен был запятой
            
            for i, token in enumerate(tokens):
                if stop:
                    break
                
                if (token[0].isupper() if token else False) and prev_comma:
                    # Слово с заглавной буквы после запятой - вероятно начало нового раздела
                    # Проверяем, не является ли это частью ключевого слова (например, "ВВП")
                    if len(token) > 1 and not any(c.islower() for c in token[1:]):
                        # Если все остальные буквы заглавные (акроним), то продолжаем
                        result_tokens.append(token)
                    else:
                        # Останавливаемся на слове с заглавной буквы после запятой
                        break
                        
                elif token.lower().startswith('jel') or token.lower().startswith('j.e.l'):
                    # начало JEL классификации
                    break
                    
                else:
                    result_tokens.append(token)
                
                # Обновляем флаг предыдущей запятой
                prev_comma = (token == ',')
            
            # Собираем результат
            keywords_result = "".join(result_tokens).strip()
            
            # Очищаем результат от лишних символов в конце
            keywords_result = re.sub(r'[,\s]+$', '', keywords_result)  # Убираем запятые и пробелы в конце
            keywords_result = re.sub(r'\s+,\s*', ', ', keywords_result)  # Нормализуем запятые
            
            return keywords_result
        
    return 'Нет ключевых слов'

## Применим функции к полученным ранее ссылкам

In [ ]:
def final_keywords_formation(data):
    for i, research in enumerate(data, 1):
        extracted_text = extract_text_from_pdf(research['link'])
        clean_text = clean_keywords_text(extracted_text)
        keywords = extract_keywords(clean_text) 
        research['keywords_total'] = keywords #список всех ключевых слов

        keyword_list = [kw.strip() for kw in re.split(r'[,;]', keywords) if kw.strip()] # разделяю список на отдельные ключевые слова
        # добавляю каждое ключевое слово в отдельный столбец
        for j, keyword in enumerate(keyword_list, 1):
            research[f'keyword_{j}'] = keyword
        
        print(f"Препринт {i}: {keywords}")
    return data

In [44]:
finally_keywords = final_keywords_formation(research_data_pdf)

Препринт 1: разрыв выпуска, потенциальный выпуск, валовой региональный продукт, HP-фильтр, фильтр Калмана, факторная декомпозиция, региональный анализ
Препринт 2: денежно-кредитная политика, малая открытая экономика, валютный рынок, таргетирование инфляции, дилемма денежно-кредитной политики, графическая модель общего равновесия, квартальная прогнозная модель, Банк России


Cannot set gray non-stroke color because /'P15' is an invalid float value


Препринт 3: постоянная эластичность замещения, CES, денежно-кредитная политика, ДКП, рынок труда


Cannot set gray non-stroke color because /'P15' is an invalid float value


Препринт 4: инфляционные ожидания, неоднородные агенты, шоки ожиданий, денежно-кредитная политика, финансовые посредники, новая кейнсианская модель, общее равновесие, неоднородность инфляционных ожиданий
Препринт 5: банки, депозиты, издержки перехода, равновесие Нэша, равновесие в безопасных стратегиях, уровень благосостояния
Препринт 6: Россия, Китай, международные деловые циклы, торговые связи, OECD TiVA, Global VAR


...

Препринт 157: Нет ключевых слов
Препринт 158: Нет ключевых слов
Препринт 159: Нет ключевых слов
Препринт 160: (униграммы), на основе которых мы самостоятельно должны «назвать» каждую из перечисленных тем
Препринт 161: Нет ключевых слов


Cannot set gray non-stroke color because /'P164' is an invalid float value
Cannot set gray non-stroke color because /'P166' is an invalid float value
Cannot set gray non-stroke color because /'P168' is an invalid float value
Cannot set gray non-stroke color because /'P171' is an invalid float value
Cannot set gray non-stroke color because /'P173' is an invalid float value
Cannot set gray non-stroke color because /'P179' is an invalid float value
Cannot set gray non-stroke color because /'P181' is an invalid float value
Cannot set gray non-stroke color because /'P183' is an invalid float value
Cannot set gray non-stroke color because /'P186' is an invalid float value
Cannot set gray non-stroke color because /'P188' is an invalid float value
Cannot set gray non-stroke color because /'P189' is an invalid float value
Cannot set gray non-stroke color because /'P193' is an invalid float value
Cannot set gray non-stroke color because /'P195' is an invalid float value
Cannot set gray non-strok

Препринт 162: Нет ключевых слов
Препринт 163: Нет ключевых слов
Препринт 164: Нет ключевых слов
Препринт 165: Нет ключевых слов


Cannot set gray non-stroke color because /'P63' is an invalid float value
Cannot set gray non-stroke color because /'P65' is an invalid float value
Cannot set gray non-stroke color because /'P68' is an invalid float value
Cannot set gray non-stroke color because /'P89' is an invalid float value
Cannot set gray non-stroke color because /'P91' is an invalid float value
Cannot set gray non-stroke color because /'P92' is an invalid float value
Cannot set gray non-stroke color because /'P94' is an invalid float value
Cannot set gray non-stroke color because /'P119' is an invalid float value
Cannot set gray non-stroke color because /'P121' is an invalid float value


Препринт 166: Нет ключевых слов
Препринт 167: Нет ключевых слов
Препринт 168: Нет ключевых слов
Препринт 169: Нет ключевых слов
Препринт 170: Нет ключевых слов
Препринт 171: Нет ключевых слов
Препринт 172: Нет ключевых слов
Препринт 173: Нет ключевых слов


# Экспортируем результаты в CSV

In [51]:
import csv

In [65]:
df = pd.DataFrame(finally_keywords)
df.to_csv('Preprints_Links_keywords', index = False, encoding='utf-8')
print("Данные экспортированы в Preprints_Links_keywords")

Данные экспортированы в Preprints_Links_keywords
